In [1]:
#Parcial de ciencia de datos
#Nathalia Gutiérrez Alarcón
#Grupo 702
#2024

import json #trabajar con datos JSON
import requests #realizar solicitudes HTTP.
from azure.cosmos import exceptions, CosmosClient, PartitionKey ##clases y funciones relacionadas con Azure Cosmos DB, manejar excepcione,interactuar con el cliente de Cosmos DB, manejar las claves de partición.
import datetime #trabajar con fechas y horas.

# Información de Azure Cosmos DB
ENDPOINT = 'https://gutierrezn.documents.azure.com:443/' 
KEY = '1qfeYtNBAnfDe003kkMq0Gehk1O5HZCZEpkBdatduyVwTatYeiMlgO9dDelm1O08ZJDmXrrGAJARACDbn5SPug=='  
DATABASE_ID = 'gutierrezn' 
CONTAINER_ID = 'COSMOS_DATABASE'  
PARTITION_KEY = '/ip'  

# Conectarse a Cosmos DB
client = CosmosClient(ENDPOINT, KEY) 
database = client.get_database_client(DATABASE_ID) 

# Verificar si la base de datos ya existe, si no crearla
database_client = None
try:
    database_client = database.read()

# Si la base de datos no existe, captura la excepción
except exceptions.CosmosResourceNotFoundError:
    # Crea la base de datos
    database_client = client.create_database(id=DATABASE_ID)

# crear el contenedor
container_definition = {'id': CONTAINER_ID, 'partition_key': {'paths': [PARTITION_KEY], 'kind': 'Hash'}}
container_options = {'offerThroughput': 400} 

# Obtener el cliente del contenedor de la base de datos (si ya existe)
container = database.get_container_client(CONTAINER_ID)
try:
    container_client = container.read()

# Si el contenedor no existe, captura la excepción 
except exceptions.CosmosResourceNotFoundError:
    container_client = database.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path=PARTITION_KEY),
                                                offer_throughput=container_options['offerThroughput'])

#Verificar si una determinada dirección IP ya está almacenada en un contenedor de Azure Cosmos DB.
def is_ip_already_stored(container, ip):
    # Verificar si la IP ya está almacenada en Cosmos DB
    query = f"SELECT * FROM c WHERE c.ip = '{ip}'"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    return len(items) > 0

#Determinar si una dirección IP específica es vulnerable a partir de una respuesta JSON 
def is_ip_vulnerable(ip, respuesta_json):
    # Verificar si hay eventos relacionados con la IP objetivo y si son vulnerables
    eventos_filtrados = [evento for evento in respuesta_json.get('events', [])
                         if evento.get('target', {}).get('ip') == ip and
                         evento.get('result', {}).get('data', {}).get('status') == 'VULNERABLE']

    return len(eventos_filtrados) > 0

#Inserta información sobre una dirección IP en un contenedor de Azure Cosmos DB
def insert_ip_info(container, ip, es_vulnerable):
    # Verificar si la IP ya está almacenada
    if is_ip_already_stored(container, ip):
        # Si la IP ya está en la base de datos, imprime un mensaje y sale de la función.
        print(f"La IP {ip} ya está almacenada en Cosmos DB.")
        return

    # Crear un documento con la información de la IP
    document = {
        'id': f'IP_{ip}',  # El identificador único del documento, se compone de 'IP_' seguido de la dirección IP.
        'ip': ip,  # La dirección IP que se está procesando.
        'vulnerable': es_vulnerable,  # Indica si la IP es vulnerable o no (booleano).
        'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Marca de tiempo actual.
    }

    # Insertar el documento en Cosmos DB
    container.create_item(body=document)  # Utiliza el cliente de contenedor para insertar el documento en Cosmos DB.
    print(f"La IP {ip} ha sido almacenada en Cosmos DB.")  # Mensaje indicando que la operación fue exitosa.

#consulta y muestra todas las direcciones IP almacenadas en un contenedor de Azure Cosmos DB.
def consultar_ips(container):
    print('\nConsulta IPs almacenadas\n')

    # Consultar todas las IPs
    items = list(container.query_items(
        query="SELECT * FROM r",
        enable_cross_partition_query=True
    ))

    for item in items:
        print(f'IP: {item["ip"]}, Vulnerable: {item["vulnerable"]}, Timestamp: {item["timestamp"]}')

#verificación de vulnerabilidad de una dirección IP, la inserción de información en una base de datos de Cosmos DB y la consulta de todas las IPs almacenadas.
def main():
    # Definir la URL de la API de BinaryEdge para buscar información sobre Bluekeep en una IP específica
    url = 'https://api.binaryedge.io/v2/query/search?query=type:bluekeep%20AND%20ip:185.198.75.90'

    # Establecer encabezados, incluido el token de autenticación (X-Key)
    headers = {'X-Key': 'a939c7ea-e183-4b2a-9138-ca69cc4bfc42'}

    # Realizar una solicitud GET a la API de BinaryEdge para obtener información sobre la IP y Bluekeep
    respuesta_api = requests.get(url, headers=headers)

    # Convertir la respuesta de la API a formato JSON
    respuesta_json = respuesta_api.json()

    # IP objetivo que estamos evaluando
    ip_objetivo = '185.198.75.90'

    # Verificar si la IP es vulnerable a Bluekeep utilizando la función is_ip_vulnerable
    es_vulnerable = is_ip_vulnerable(ip_objetivo, respuesta_json)

    # Imprimir el resultado basado en si la IP es vulnerable o no
    if es_vulnerable:
        print(f"La IP {ip_objetivo} es vulnerable a Bluekeep.")
    else:
        print(f"La IP {ip_objetivo} no es vulnerable a Bluekeep.")

    # Insertar información en Cosmos DB si la IP no está almacenada previamente
    insert_ip_info(container, ip_objetivo, es_vulnerable)

    # Consultar todas las IPs almacenadas en Cosmos DB
    consultar_ips(container)

# La siguiente línea ejecuta la función main() solo si este script es ejecutado como el programa principal
if __name__ == '__main__':
    main()



La IP 185.198.75.90 es vulnerable a Bluekeep.
La IP 185.198.75.90 ya está almacenada en Cosmos DB.

Consulta IPs almacenadas

IP: 185.198.75.90, Vulnerable: True, Timestamp: 2024-03-06 09:48:58
